In [1]:
# import sys
import json
import uuid
from tqdm.notebook import tqdm
import datetime 
from openpyxl import Workbook
import os

# path is broken on my machine, so I leave this here for myself :)
# sys.path.append('/Users/veronicasmilga/Desktop/Tübingen/MSE/Project_MSE/')

from db.DocumentEntry import DocumentEntry
from db.DocumentRepository import DocumentRepository
from data_retrieval.Crawler import Crawler

/Users/danielwps/Documents/Private/Github/Project_MSE/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to /Users/danielwps/Document
[nltk_data]     s/Private/Github/Project_MSE/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/danielwps/Documents/Pr
[nltk_data]     ivate/Github/Project_MSE/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# frontier now is in a separate file
with open("../frontier.json", "r") as file:
    frontier = json.load(file)

In [3]:
SAVE_TO_DATABASE = True

If "SAVE_TO_DATABASE" is set to True, we have to setup docker before that

In [4]:
# Connect to the database by exec docker compose in your terminal. This exec. a terminal command using Python
if SAVE_TO_DATABASE:
    os.system("""
    docker compose down;
    docker compose up -d --build db;
    """)

 Container project_mse-db-1  Stopping
 Container project_mse-db-1  Stopped
 Container project_mse-db-1  Removing
 Container project_mse-db-1  Removed
 Network project_mse_default  Removing
 Network project_mse_default  Removed


#0 building with "desktop-linux" instance using docker driver

#1 [db internal] load build definition from Dockerfile
#1 transferring dockerfile: 122B done
#1 DONE 0.0s

#2 [db internal] load metadata for docker.io/library/postgres:latest
#2 DONE 0.0s

#3 [db internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [db 1/2] FROM docker.io/library/postgres:latest
#4 DONE 0.0s

#5 [db internal] load build context
#5 transferring context: 30B done
#5 DONE 0.0s

#6 [db 2/2] COPY dump.sql /docker-entrypoint-initdb.d/
#6 CACHED

#7 [db] exporting to image
#7 exporting layers done
#7 writing image sha256:3a71a465c2e6efe5bccb23028b9f1eb04b1b898db39add2d7116c1e6338e75ac done
#7 naming to docker.io/library/project_mse-db done
#7 DONE 0.0s


 Network project_mse_default  Creating
 Network project_mse_default  Created
 Container project_mse-db-1  Creating
 Container project_mse-db-1  Created
 Container project_mse-db-1  Starting
 Container project_mse-db-1  Started


In [8]:
# initialising the database
if SAVE_TO_DATABASE:
    documentRepository = DocumentRepository()

# initialising the Excel backup (if sth goes wrong with the database)
wb = Workbook()
ws = wb.active
ws.title = "Crawled Data"
headers = ["id", "url", "title", "headings", "page_text", "keywords", "accessed_timestamp", "internal_links", "external_links"]
ws.append(headers)

SC: Connected to the db. Now you can go and build the best search engine around!


Just in case you set "SAVE_TO_DATABASE" = True and you did not get
`SC: Connected to the db. Now you can go and build the best search engine around!`, just **run the cell again**. 

Otherwise, just continue :)

In [9]:
now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

max_pages = 1
max_steps_per_domain = 10
timeout = 10

# Crawler is an iterator now, to handle info generated on-the-fly and save it immediately
crawler = Crawler(frontier, max_pages, max_steps_per_domain, timeout)

for scraped_webpage_info in tqdm(crawler, total=max_pages):
    # save one crawled page to excel file
    row = [
        str(uuid.uuid4()),
        scraped_webpage_info["url"],
        scraped_webpage_info["title"],
        str(scraped_webpage_info["headings"]),
        scraped_webpage_info["page_text"],
        str(scraped_webpage_info["keywords"]),
        scraped_webpage_info["accessed_timestamp"],
        str(scraped_webpage_info["internal_links"]),
        str(scraped_webpage_info["external_links"])
    ]
    ws.append(row)
    wb.save(f"./data/crawled_data_backup_{now}.xlsx")

    if SAVE_TO_DATABASE:
        # save one crawled page to database
        document = DocumentEntry(
            url=scraped_webpage_info["url"],
            title=scraped_webpage_info["title"],
            headings=scraped_webpage_info["headings"],
            page_text=scraped_webpage_info["page_text"], 
            keywords=scraped_webpage_info["keywords"],
            accessed_timestamp=scraped_webpage_info["accessed_timestamp"],
            internal_links=scraped_webpage_info["internal_links"],
            external_links=scraped_webpage_info["external_links"],
            id=uuid.uuid4()
            )
        documentRepository.saveDocument(document)

  0%|          | 0/1 [00:00<?, ?it/s]

Pages crawled: 0. Pages left: 1.
Crawling: https://uni-tuebingen.de/en/
Indexing...
Indexed url https://uni-tuebingen.de/en/ with title `Home | University of Tübingen` successfully.
Filtered out invalid internal link: tel:+4970712974444
Filtered out invalid internal link: javascript:linkTo_UnCryptMailto(%27ocknvq%2CuvwfkgtgpfgpugmtgvctkcvBbx0wpk%5C%2Fvwgdkpigp0fg%27);
Filtered out invalid internal link: tel:+497071290
SC: Saved document.
Reached the maximum number of pages to crawl.


If you want to access documents in the database. **CAUTION: LONG OUTPUT**. After we will shut down docker again

In [ ]:
if SAVE_TO_DATABASE:
    allDocuments = documentRepository.loadAllDocuments()
    print(allDocuments)
    
    # df: Finally let's
    os.system("""
docker compose down;
""")

Otherwise, if you just want to take a look at the data, go to `exp/data` and find the Excel file with timestamp corresponding to the time you ran the Crawler :)